In [1]:
import aliases

In [2]:
from everest import incision

In [3]:
incision.Incisable()

TypeError: Can't instantiate abstract class Incisable with abstract method __contains__

In [ ]:
class Metric(incision.Incisable):

    ...


class Ordered(Metric):
    

In [ ]:
class MyClass(incision.Incisable, collabc.Sequence):
    ...

In [ ]:
from collections import abc as collabc

In [ ]:
Metric.Incision